In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import html
%matplotlib inline
from sklearn import linear_model

In [46]:
train_path ='https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv'
test_path = 'https://raw.githubusercontent.com/gurkpet/Thinkful-Lessons/8b1bfca035fc0505813326ee7ceb370d514e1884/Thinkful%202.5.1%20-%20Test%20Statistics/Table_8_Offenses_Known_to_Law_Enforcement_by_New_York_by_City_2014.txt'
train_data = pd.read_csv(train_path, skiprows=4, skipfooter=3, engine='python', thousands=",")
test_data = pd.DataFrame()
test_data = pd.read_table(test_path, skiprows=0, skipfooter=7, engine='python', thousands=",")

test_data.drop(test_data.columns[len(test_data.columns)-1], axis=1, inplace=True)

In [47]:
train_data = train_data.drop('Rape\n(revised\ndefinition)1', 1)
test_data = test_data.drop('Rape\n(revised\ndefinition)1', 1)

In [48]:
train_data = train_data.fillna(train_data.median(), inplace = True)
test_data = test_data.fillna(test_data.median(), inplace = True)

In [49]:
column_heads= ['City',
               'Population',
              'Violent_Crime',
               'Murder',
               'Rape',
               'Robbery',
               'Aggravated_Assult',
               'Property_Crime',
               'Burglary',
               'Larceny_Theft',
               'Motor_Vehicle_Theft',
               'Arson'
              ]

In [50]:
train_data.columns = [column_heads]
test_data.columns = [column_heads]

In [51]:
(test_data.isnull().sum()/len(test_data))*100

City                   0.0
Population             0.0
Violent_Crime          0.0
Murder                 0.0
Rape                   0.0
Robbery                0.0
Aggravated_Assult      0.0
Property_Crime         0.0
Burglary               0.0
Larceny_Theft          0.0
Motor_Vehicle_Theft    0.0
Arson                  0.0
dtype: float64

In [52]:
(train_data.isnull().sum()/len(train_data))*100

City                   0.0
Population             0.0
Violent_Crime          0.0
Murder                 0.0
Rape                   0.0
Robbery                0.0
Aggravated_Assult      0.0
Property_Crime         0.0
Burglary               0.0
Larceny_Theft          0.0
Motor_Vehicle_Theft    0.0
Arson                  0.0
dtype: float64

In [53]:
print(test_data.shape, '<--- test data shape')
print(train_data.shape, '<--- train data shape')

(369, 12) <--- test data shape
(348, 12) <--- train data shape


In [56]:
test_data['City'] = test_data.City.str.strip('5 4,')
test_data = test_data.dropna()

In [57]:
train_data['IncidentsOfPropertyCrime'] = (train_data.Burglary.astype(int) + train_data.Robbery +
    train_data.Larceny_Theft + train_data.Motor_Vehicle_Theft)

In [65]:
train_data.describe()

,Population,Violent_Crime,Murder,Rape,Robbery,Aggravated_Assult,Property_Crime,Burglary,Larceny_Theft,Motor_Vehicle_Theft,Arson,IncidentsOfPropertyCrime
count,3.480000e+02,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000
mean,4.003763e+04,201.594828,1.566092,5.864943,72.902299,121.261494,792.606322,119.683908,637.017241,35.905172,1.005747,865.508621
std,4.500374e+05,2815.268504,18.303673,60.425452,1031.032873,1706.131730,7659.724746,924.948789,6346.054451,403.423826,7.884612,8689.155368
min,5.260000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.003000e+03,2.000000,0.000000,0.000000,0.000000,1.000000,40.500000,6.000000,31.000000,0.000000,0.000000,40.500000
50%,7.233500e+03,6.000000,0.000000,0.000000,1.000000,4.000000,112.500000,17.500000,94.000000,2.000000,0.000000,114.500000
75%,1.842750e+04,22.000000,0.000000,2.000000,5.000000,14.000000,341.000000,51.250000,287.250000,7.000000,0.000000,346.500000
max,8.396126e+06,52384.000000,335.000000,1112.000000,19170.000000,31767.000000,141971.000000,16606.000000,117931.000000,7434.000000,132.000000,161141.000000


In [70]:
train_data = train_data.drop(train_data[train_data.Population > 1800000].index)
test_data = test_data.drop(test_data[test_data.Population > 1800000].index)

In [71]:
test_data['IncidentsOfPropertyCrime'] = (test_data.Burglary.astype(int) + test_data.Robbery +
    test_data.Larceny_Theft + test_data.Motor_Vehicle_Theft)

In [72]:
regr = linear_model.LinearRegression()
Y = train_data['IncidentsOfPropertyCrime'].values.reshape(-1, 1)
X = train_data[['Population', 'Burglary','Robbery','Larceny_Theft']]
regr.fit(X, Y)

print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[  1.72009833e-04   1.10117274e+00   1.44884797e+00   9.91695801e-01]]

Intercept: 
 [-0.99847814]

R-squared:
0.9999496577


In [73]:
regr = linear_model.LinearRegression()
Y = test_data['IncidentsOfPropertyCrime'].values.reshape(-1, 1)
X = test_data[['Population', 'Burglary','Robbery','Larceny_Theft']]
regr.fit(X, Y)

print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[  7.68857994e-05   1.16943252e+00   1.40152997e+00   9.90081927e-01]]

Intercept: 
 [-1.30761808]

R-squared:
0.999928886974
